In [1]:
import time
import math
import random
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
%matplotlib inline

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'
tf.test.is_gpu_available()

True

# Configs & Hyperparameters

In [3]:
# batch size
batch_size = 32
# validation split
valid_size = .1
# test split
test_size = 0.2

img_dim = (200, 200)

# paths
cache_path = 'cache'
data_path = 'data-filtered'

# Data Loading

In [4]:
import os

n_birds = 10 # use ten kinds of birds first
bird_file_map = {}

# return array of bird names
birdList = sorted(os.listdir(data_path))
loadedImages = []

n_images = 0
n_birds_loaded = 0
for b in birdList:
    if n_birds_loaded >= n_birds:
        break
    print("Loading images for '" + b + "'")
    curdir = os.path.join(data_path, b)
    if not os.path.isdir(curdir):
        continue
    img_files = os.listdir(curdir)
    
    filenames = [os.path.join(curdir, f) for f in img_files]
    n_f = len(filenames)
    if n_f > 0: # use data only if more than xx images are found
        bird_file_map[b] = filenames
        print(n_f, "images loaded for '" + b + "'")
        n_birds_loaded += 1
        n_images += n_f
    else:
        print("Not enough data for '" + b + "'")

Loading images for '.ipynb_checkpoints'
Not enough data for '.ipynb_checkpoints'
Loading images for 'Aberrant Bush-Warbler'
271 images loaded for 'Aberrant Bush-Warbler'
Loading images for 'Ala Shan Redstart'
401 images loaded for 'Ala Shan Redstart'
Loading images for 'Aleutian Tern'
282 images loaded for 'Aleutian Tern'
Loading images for 'Altai Snowcock'
244 images loaded for 'Altai Snowcock'
Loading images for 'American Wigeon'
486 images loaded for 'American Wigeon'
Loading images for 'Arctic Warbler'
329 images loaded for 'Arctic Warbler'
Loading images for 'Ashy Bulbul'
310 images loaded for 'Ashy Bulbul'
Loading images for 'Ashy Drongo'
412 images loaded for 'Ashy Drongo'
Loading images for 'Ashy Minivet'
443 images loaded for 'Ashy Minivet'
Loading images for 'Ashy Wood Pigeon'
335 images loaded for 'Ashy Wood Pigeon'


In [5]:
import cv2
n_channels = 3
X = np.zeros((n_images, img_dim[0], img_dim[1], n_channels))
labels = []

i = 0
for k, v in bird_file_map.items():
    for file in v:
        try:
            im = cv2.imread(file)
            if im is None or im.shape[0] < img_dim[0] or im.shape[1] < img_dim[1]:
                continue
            shape = im.shape
            assert len(shape) == 3 # width, length and color channels
            assert shape[-1] == 3 # rgb, three channels
            
            # resizing
            im = cv2.resize(src=im, dsize=img_dim, interpolation=cv2.INTER_LINEAR)
            # Gaussian blurring
            # im = cv2.GaussianBlur(im, (5, 5), 0)
            X[i] = np.asarray(im)
            labels.append(k)
            i += 1
        except IOError:
            continue
n_images = len(labels)
X = X[:n_images, ]
del i

In [6]:
labels_unique = list(set(labels))

Y = np.zeros((n_images, 1))
for i in range(n_images):
    Y[i, 0] = labels_unique.index(labels[i])

In [7]:
print(X.shape)
print(Y.shape)
print(np.min(X), np.max(X))

(2138, 200, 200, 3)
(2138, 1)
0.0 255.0


## Normalization

In [8]:
for i in range(X.shape[0]):
    m = np.min(X[i,])
    X[i] = (X[i,] - m) / (np.max(X[i,]) - m)

print(np.min(X), np.max(X))

0.0 1.0


# Reducing memory size

In [9]:
# Function to reduce the numpy array size
def reduce_mem_usage(a, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = a.nbytes / 1024**2
    dtype = a.dtype
    if dtype in numerics:
        c_min = a.min()
        c_max = a.max()
        if str(dtype)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                a = a.astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                a = a.astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                a = a.astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                a = a.astype(np.int64)  
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                a = a.astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                a = a.astype(np.float32)
            else:
                a = a.astype(np.float64)
    end_mem = a.nbytes / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return a

X = reduce_mem_usage(X)
Y = reduce_mem_usage(Y)

Mem. usage decreased to 489.35 Mb (75.0% reduction)
Mem. usage decreased to  0.00 Mb (75.0% reduction)


# Data Splitting

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size)

# Data Augmentation

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_gen_batch_size = 32
image_gen_train = ImageDataGenerator(
                    rotation_range=45,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    horizontal_flip=True,
                    shear_range=0.2,
                    zoom_range=0.1)

image_gen_flow = image_gen_train.flow(X_train, y_train, batch_size=img_gen_batch_size)
X_added = np.zeros((len(image_gen_flow) * img_gen_batch_size, *(X_train.shape[1:])))
Y_added = np.zeros((len(image_gen_flow) * img_gen_batch_size, *(y_train.shape[1:])))
flow_len = len(image_gen_flow)

n_added = 0
for i in range(0, flow_len):
    X_batch = image_gen_flow[i][0]
    img_gen_batch_size = X_batch.shape[0]
    X_added[i * img_gen_batch_size: (i + 1) * img_gen_batch_size,] = X_batch
    Y_added[i * img_gen_batch_size: (i + 1) * img_gen_batch_size,] = image_gen_flow[i][1]
    n_added += img_gen_batch_size
print('data augmentation sucessful, {} new images in total were added'.format(n_added))
del flow_len
del X_batch
del img_gen_batch_size

data augmentation sucessful, 1539 new images in total were added


In [12]:
X_train = np.vstack([X_train, X_added])
y_train = np.vstack([y_train, Y_added])
del X_added
del Y_added

In [13]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_valid = to_categorical(y_valid)
print(X_train.shape, '|', X_test.shape)
print(y_train.shape, '|', y_test.shape, '|', y_valid.shape)

(3107, 200, 200, 3) | (428, 200, 200, 3)
(3107, 10) | (428, 10) | (171, 10)


# Building CNN

In [16]:
n_classes = n_birds

model = models.Sequential()
model.add(layers.Conv2D(64, 3, strides=1, input_shape=(img_dim[0], img_dim[1], 3)))
model.add(layers.Conv2D(64, 3, strides=1))
model.add(layers.MaxPooling2D(2, strides=2))

model.add(layers.Conv2D(128, 3, strides=1))
model.add(layers.Conv2D(128, 3, strides=1))
model.add(layers.MaxPooling2D(2, strides=2))

model.add(layers.Conv2D(256, 3, strides=1))
model.add(layers.Conv2D(256, 3, strides=1))
model.add(layers.Conv2D(256, 3, strides=1))
model.add(layers.MaxPooling2D(2, strides=2))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(n_classes, activation='relu'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 198, 198, 64)      1792      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 196, 196, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 96, 96, 128)       73856     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 94, 94, 128)       147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 47, 47, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 45, 45, 256)      

# Training

In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20)

Train on 3107 samples
Epoch 1/20
3107/3107 [==============================] - 21s 7ms/sample - loss: 8.7213 - accuracy: 0.1175
Epoch 2/20
3107/3107 [==============================] - 16s 5ms/sample - loss: 8.5887 - accuracy: 0.1271
Epoch 3/20
3107/3107 [==============================] - 16s 5ms/sample - loss: 6.7485 - accuracy: 0.1207
Epoch 4/20
3107/3107 [==============================] - 16s 5ms/sample - loss: 5.7078 - accuracy: 0.1201
Epoch 5/20
3107/3107 [==============================] - 16s 5ms/sample - loss: 5.7057 - accuracy: 0.1316
Epoch 6/20
3107/3107 [==============================] - 17s 5ms/sample - loss: 5.7039 - accuracy: 0.1316
Epoch 7/20
3107/3107 [==============================] - 16s 5ms/sample - loss: 5.7025 - accuracy: 0.1423
Epoch 8/20
3107/3107 [==============================] - 16s 5ms/sample - loss: 5.6964 - accuracy: 0.1558
Epoch 9/20
3107/3107 [==============================] - 16s 5ms/sample - loss: 5.6773 - accuracy: 0.1690
Epoch 10/20
3107/3107 [==========

# Evaluation

In [18]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

428/1 - 1s - loss: 4.7592 - accuracy: 0.2407


In [19]:
# Save the weights
model.save('model.h5')